In [112]:
import requests
import lxml.html
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from dateutil.relativedelta import relativedelta
import urllib.request

In [113]:
obs_stations = pd.read_csv("obs_stations.csv", encoding='shift_jis',header=1)
obs_stations = obs_stations.query('ed_y > 2019')
obs_stations = obs_stations.query('気温 == "Y"')
obs_stations = obs_stations.reset_index(drop=True)

In [114]:
obs_stations.head(10)

,地点コード,地点,地点カナ,府県番号,区分,緯度_度,緯度_分,緯度,経度_度,経度_分,...,ed_m,ed_d,観測終了日,備考1,備考2,備考3,備考4,備考5,観測所番号1,観測所番号2
0,47401,稚内,ワツカナイ,11,S,45,24.9,45.4150,141,40.7,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11016.0,11903.0
1,2,沓形,クツガタ,11,A,45,10.6,45.1767,141,8.3,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11151.0,NaN
2,3,浜頓別,ハマトンベツ,11,A,45,7.5,45.1250,142,21.0,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11206.0,NaN
3,47402,北見枝幸,キタミエサシ,11,S,44,56.4,44.9400,142,35.1,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11291.0,11917.0
4,5,歌登,ウタノボリ,11,A,44,50.4,44.8400,142,28.8,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11316.0,11920.0
5,1051,中頓別,ナカトンベツ,11,A,44,57.9,44.9650,142,16.8,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11276.0,11915.0
6,1054,豊富,トヨトミ,11,A,45,6.4,45.1067,141,45.9,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11176.0,11910.0
7,1203,沼川,ヌマカワ,11,A,45,14.9,45.2483,141,51.1,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11121.0,11905.0
8,1284,宗谷岬,ソウヤミサキ,11,A,45,31.2,45.5200,141,56.1,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11001.0,NaN
9,1285,浜鬼志別,ハマオニシベツ,11,A,45,20.1,45.3350,142,10.2,...,99,99,NaN,NaN,NaN,NaN,NaN,NaN,11076.0,11900.0


In [118]:
#2020年用コード
start_time = datetime.date(2020, 1, 1)
end_time   = datetime.date(2020, 12, 31)

now=time.strftime('%Y/%m/%d %H:%M:%S')
print(now)
Temp_df = pd.DataFrame(index=[], columns=['地点コード','地点','地点カナ','都道府県番号','区分','月','緯度','経度','降水量合計','平均気温','平均湿度','平均風速','日照時間合計','降雪日数'])
Temp_df_copy1 = Temp_df.copy()
Temp_df_copy2 = Temp_df.copy()
for i in range(len(obs_stations)):
    Temp_df_copy2 = Temp_df_copy1.copy()
    month = start_time
    while month < end_time:
        Temp_df_copy2.loc[month,'地点コード']=obs_stations['地点コード'][i]
        Temp_df_copy2.loc[month,'地点']=obs_stations['地点'][i]
        Temp_df_copy2.loc[month,'地点カナ']=obs_stations['地点カナ'][i]
        Temp_df_copy2.loc[month,'都道府県番号']=obs_stations['府県番号'][i]
        Temp_df_copy2.loc[month,'区分']=obs_stations['区分'][i]
        Temp_df_copy2.loc[month,'月']=month.month
        Temp_df_copy2.loc[month,'緯度']=obs_stations['緯度'][i]
        Temp_df_copy2.loc[month,'経度']=obs_stations['経度'][i]
        month += relativedelta(months=1)
    Temp_df = pd.concat([Temp_df,Temp_df_copy2], axis=0,ignore_index=True)  

def str2float(weather_data):
    try:
        return float(weather_data)
    except:
        return -100

def safe_split_string(element, default='NaN'):
    if element is not None and element.string is not None:
        return element.string.split(' ')[0]
    else:
        return default

# 各観測所のデータを取得
Temp_df_result = Temp_df.copy()
num = 0
for index, value in obs_stations.iterrows():
    date_month = start_time

    url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_%s1.php?prec_no=%d&block_no=%04d&year=%d&month=&day=&view=" \
        % (str.lower(obs_stations['区分'].loc[index]), value['府県番号'], value['地点コード'], date_month.year)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find("table", {"class": "data2_s"})

    if trs is None:
        continue

    print(f"\r{index, obs_stations['地点'].loc[index], obs_stations['地点コード'].loc[index], date_month.year}", end="")
    if obs_stations['区分'].loc[index] == 'S':
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result.loc[num, '降水量合計'] = str2float(safe_split_string(tds[3]))
            Temp_df_result.loc[num, '平均気温'] = str2float(safe_split_string(tds[7]))
            Temp_df_result.loc[num, '平均湿度'] = str2float(safe_split_string(tds[12]))
            Temp_df_result.loc[num, '平均風速'] = str2float(safe_split_string(tds[14]))
            Temp_df_result.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[19]))
            Temp_df_result.loc[num, '降雪日数'] = str2float(safe_split_string(tds[25]))
            date_month += relativedelta(months=1)
            num += 1
    else:
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result.loc[num, '降水量合計'] = str2float(safe_split_string(tds[1]))
            Temp_df_result.loc[num, '平均気温'] = str2float(safe_split_string(tds[5]))
            Temp_df_result.loc[num, '平均湿度'] = str2float(safe_split_string(tds[10]))
            Temp_df_result.loc[num, '平均風速'] = str2float(safe_split_string(tds[12]))
            Temp_df_result.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[17]))
            Temp_df_result.loc[num, '降雪日数'] = None
            date_month += relativedelta(months=1)
            num += 1
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    time.sleep(3)

Temp_202001_202012_by_month=Temp_df_result

Temp_df_result.to_csv("2020_by_month_data.csv", encoding="shift_jis",
                      index=False,header=True)

2024/06/20 20:24:48
(0, '稚内', 47401, 2020)2024/06/20 20:25:08
(1, '沓形', 2, 2020)2024/06/20 20:25:11
(2, '浜頓別', 3, 2020)2024/06/20 20:25:15
(3, '北見枝幸', 47402, 2020)2024/06/20 20:25:18
(4, '歌登', 5, 2020)2024/06/20 20:25:21
(5, '中頓別', 1051, 2020)2024/06/20 20:25:24
(6, '豊富', 1054, 2020)2024/06/20 20:25:28
(7, '沼川', 1203, 2020)2024/06/20 20:25:31
(8, '宗谷岬', 1284, 2020)2024/06/20 20:25:34
(9, '浜鬼志別', 1285, 2020)2024/06/20 20:25:38
(10, '本泊', 1512, 2020)2024/06/20 20:25:41
(11, '声問', 1528, 2020)2024/06/20 20:25:44
(12, '礼文', 1546, 2020)2024/06/20 20:25:48
(13, '美深', 6, 2020)2024/06/20 20:25:51
(14, '名寄', 8, 2020)2024/06/20 20:25:54
(15, '下川', 10, 2020)2024/06/20 20:25:57
(16, '士別', 11, 2020)2024/06/20 20:26:01
(17, '上川', 15, 2020)2024/06/20 20:26:04
(18, '旭川', 47407, 2020)2024/06/20 20:26:07
(19, '富良野', 21, 2020)2024/06/20 20:26:10
(20, '朝日', 1047, 2020)2024/06/20 20:26:14
(21, '比布', 1048, 2020)2024/06/20 20:26:17
(22, '美瑛', 1052, 2020)2024/06/20 20:26:20
(23, '和寒', 1055, 2020)2024/06/20 20:

(194, '脇野沢', 1124, 2020)2024/06/20 20:35:47
(195, '青森大谷', 1495, 2020)2024/06/20 20:35:51
(196, '能代', 183, 2020)2024/06/20 20:35:54
(197, '鷹巣', 184, 2020)2024/06/20 20:35:57
(198, '鹿角', 185, 2020)2024/06/20 20:36:01
(199, '湯瀬', 186, 2020)2024/06/20 20:36:04
(200, '五城目', 188, 2020)2024/06/20 20:36:07
(201, '秋田', 47582, 2020)2024/06/20 20:36:10
(202, '岩見三内', 192, 2020)2024/06/20 20:36:14
(203, '角館', 193, 2020)2024/06/20 20:36:17
(204, '田沢湖', 194, 2020)2024/06/20 20:36:20
(205, '大曲', 195, 2020)2024/06/20 20:36:24
(206, '本荘', 196, 2020)2024/06/20 20:36:27
(207, '横手', 198, 2020)2024/06/20 20:36:30
(208, '象潟', 199, 2020)2024/06/20 20:36:34
(209, '矢島', 200, 2020)2024/06/20 20:36:37
(210, '湯沢', 202, 2020)2024/06/20 20:36:40
(211, '湯の岱', 204, 2020)2024/06/20 20:36:43
(212, '大館', 912, 2020)2024/06/20 20:36:46
(213, '大潟', 1035, 2020)2024/06/20 20:36:50
(214, '男鹿', 1036, 2020)2024/06/20 20:36:53
(215, '八森', 1043, 2020)2024/06/20 20:36:56
(216, '大正寺', 1050, 2020)2024/06/20 20:36:59
(217, '東由利', 1117

(385, '八王子', 366, 2020)2024/06/20 20:46:18
(386, '東京', 47662, 2020)2024/06/20 20:46:21
(387, '江戸川臨海', 370, 2020)2024/06/20 20:46:25
(388, '羽田', 371, 2020)2024/06/20 20:46:28
(389, '大島', 47675, 2020)2024/06/20 20:46:31
(390, '三宅島', 47677, 2020)2024/06/20 20:46:34
(391, '八丈島', 47678, 2020)2024/06/20 20:46:38
(392, '青梅', 1001, 2020)2024/06/20 20:46:41
(393, '練馬', 1002, 2020)2024/06/20 20:46:44
(394, '府中', 1133, 2020)2024/06/20 20:46:47
(395, '父島', 47971, 2020)2024/06/20 20:46:51
(396, '大島北ノ山', 1467, 2020)2024/06/20 20:46:54
(397, '八重見ヶ原', 1468, 2020)2024/06/20 20:46:57
(398, '三宅坪田', 1506, 2020)2024/06/20 20:47:00
(399, '新島', 1522, 2020)2024/06/20 20:47:03
(400, '神津島', 1523, 2020)2024/06/20 20:47:07
(401, '南鳥島', 47991, 2020)2024/06/20 20:47:10
(402, '香取', 375, 2020)2024/06/20 20:47:13
(403, '我孫子', 376, 2020)2024/06/20 20:47:17
(404, '成田', 378, 2020)2024/06/20 20:47:20
(405, '銚子', 47648, 2020)2024/06/20 20:47:23
(406, '千葉', 47682, 2020)2024/06/20 20:47:27
(407, '茂原', 381, 2020)2024/06/20 20

(575, '栢野', 571, 2020)2024/06/20 20:56:37
(576, '白山吉野', 973, 2020)2024/06/20 20:56:40
(577, '小松', 1324, 2020)2024/06/20 20:56:44
(578, '三井', 1544, 2020)2024/06/20 20:56:47
(579, '福井', 47616, 2020)2024/06/20 20:56:50
(580, '大野', 573, 2020)2024/06/20 20:56:54
(581, '今庄', 577, 2020)2024/06/20 20:56:57
(582, '敦賀', 47631, 2020)2024/06/20 20:57:00
(583, '小浜', 579, 2020)2024/06/20 20:57:04
(584, '美浜', 1010, 2020)2024/06/20 20:57:07
(585, '三国', 1071, 2020)2024/06/20 20:57:11
(586, '勝山', 1226, 2020)2024/06/20 20:57:14
(587, '越廼', 1316, 2020)2024/06/20 20:57:17
(588, '春江', 1537, 2020)2024/06/20 20:57:20
(589, '今津', 580, 2020)2024/06/20 20:57:24
(590, '彦根', 47761, 2020)2024/06/20 20:57:27
(591, '大津', 586, 2020)2024/06/20 20:57:30
(592, '土山', 587, 2020)2024/06/20 20:57:33
(593, '東近江', 963, 2020)2024/06/20 20:57:37
(594, '信楽', 964, 2020)2024/06/20 20:57:40
(595, '長浜', 976, 2020)2024/06/20 20:57:43
(596, '南小松', 1083, 2020)2024/06/20 20:57:47
(597, '米原', 1524, 2020)2024/06/20 20:57:50
(598, '宮津', 589

(766, '秋吉台', 769, 2020)2024/06/20 21:07:04
(767, '広瀬', 771, 2020)2024/06/20 21:07:07
(768, '豊田', 772, 2020)2024/06/20 21:07:11
(769, '山口', 47784, 2020)2024/06/20 21:07:14
(770, '防府', 775, 2020)2024/06/20 21:07:17
(771, '下松', 776, 2020)2024/06/20 21:07:21
(772, '下関', 47762, 2020)2024/06/20 21:07:24
(773, '宇部', 778, 2020)2024/06/20 21:07:28
(774, '安下庄', 779, 2020)2024/06/20 21:07:31
(775, '玖珂', 913, 2020)2024/06/20 21:07:34
(776, '油谷', 939, 2020)2024/06/20 21:07:37
(777, '岩国', 940, 2020)2024/06/20 21:07:40
(778, '柳井', 942, 2020)2024/06/20 21:07:44
(779, '八幡', 780, 2020)2024/06/20 21:07:47
(780, '行橋', 782, 2020)2024/06/20 21:07:50
(781, '飯塚', 47809, 2020)2024/06/20 21:07:54
(782, '前原', 785, 2020)2024/06/20 21:07:57
(783, '福岡', 47807, 2020)2024/06/20 21:08:01
(784, '朝倉', 788, 2020)2024/06/20 21:08:04
(785, '久留米', 790, 2020)2024/06/20 21:08:07
(786, '黒木', 792, 2020)2024/06/20 21:08:10
(787, '大牟田', 793, 2020)2024/06/20 21:08:13
(788, '宗像', 943, 2020)2024/06/20 21:08:17
(789, '添田', 1046, 2020

PermissionError: [Errno 13] Permission denied: '2020_by_month_data.csv'

In [121]:
#2019年用コード
start_time_2019 = datetime.date(2019, 1, 1)
end_time_2019   = datetime.date(2019, 12, 31)

#現在の時刻を表示
now=time.strftime('%Y/%m/%d %H:%M:%S')
print(now)

Temp_df = pd.DataFrame(index=[], columns=['地点コード','地点','地点カナ','都道府県番号','区分','月','緯度','経度','降水量合計','平均気温','平均湿度','平均風速','日照時間合計','降雪日数'])
Temp_df_2019 = Temp_df.copy()
Temp_df_copy1 = Temp_df.copy()
Temp_df_copy2 = Temp_df.copy()
    
#2019_11から2019_12までのデータフレーム
for i in range(len(obs_stations)):
    Temp_df_copy2 = Temp_df_copy1.copy()
    month = start_time_2019
    while month < end_time_2019:
        Temp_df_copy2.loc[month,'地点コード']=obs_stations['地点コード'][i]
        Temp_df_copy2.loc[month,'地点']=obs_stations['地点'][i]
        Temp_df_copy2.loc[month,'地点カナ']=obs_stations['地点カナ'][i]
        Temp_df_copy2.loc[month,'都道府県番号']=obs_stations['府県番号'][i]
        Temp_df_copy2.loc[month,'区分']=obs_stations['区分'][i]
        Temp_df_copy2.loc[month,'月']=month.month
        Temp_df_copy2.loc[month,'緯度']=obs_stations['緯度'][i]
        Temp_df_copy2.loc[month,'経度']=obs_stations['経度'][i]
        Temp_df_copy2.loc[month,'降水量合計':] = -100.0
        month += relativedelta(months=1)
    Temp_df_2019 = pd.concat([Temp_df_2019,Temp_df_copy2], axis=0,ignore_index=True) 

def str2float(weather_data):
    try:
        return float(weather_data)
    except:
        return -100

def safe_split_string(element, default='NaN'):
    if element is not None and element.string is not None:
        return element.string.split(' ')[0]
    else:
        return default

# 各観測所のデータを取得
Temp_df_result_2019 = Temp_df_2019.copy()

num = 0
for index, value in obs_stations.iterrows():
    date_month = start_time_2019

    url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_%s1.php?prec_no=%d&block_no=%04d&year=%d&month=&day=&view=" \
        % (str.lower(obs_stations['区分'].loc[index]), value['府県番号'], value['地点コード'], date_month.year)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find("table", {"class": "data2_s"})

    if trs is None:
        continue

    print(f"\r{index, obs_stations['地点'].loc[index], obs_stations['地点コード'].loc[index], date_month.year}", end="")
    if obs_stations['区分'].loc[index] == 'S':
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2019.loc[num, '降水量合計'] = str2float(safe_split_string(tds[3]))
            Temp_df_result_2019.loc[num, '平均気温'] = str2float(safe_split_string(tds[7]))
            Temp_df_result_2019.loc[num, '平均湿度'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2019.loc[num, '平均風速'] = str2float(safe_split_string(tds[14]))
            Temp_df_result_2019.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[19]))
            Temp_df_result_2019.loc[num, '降雪日数'] = str2float(safe_split_string(tds[25]))
            date_month += relativedelta(months=1)
            num += 1
    else:
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2019.loc[num, '降水量合計'] = str2float(safe_split_string(tds[1]))
            Temp_df_result_2019.loc[num, '平均気温'] = str2float(safe_split_string(tds[5]))
            Temp_df_result_2019.loc[num, '平均湿度'] = str2float(safe_split_string(tds[10]))
            Temp_df_result_2019.loc[num, '平均風速'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2019.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[17]))
            Temp_df_result_2019.loc[num, '降雪日数'] = None
            date_month += relativedelta(months=1)
            num += 1
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    time.sleep(3)

Temp_201901_201912_by_month=Temp_df_result_2019

Temp_201901_201912_by_month.to_csv("2019_by_month_data.csv", encoding="shift_jis",
                      index=False,header=True)

2024/06/20 21:41:04
(0, '稚内', 47401, 2019)2024/06/20 21:41:18
(1, '沓形', 2, 2019)2024/06/20 21:41:22
(2, '浜頓別', 3, 2019)2024/06/20 21:41:25
(3, '北見枝幸', 47402, 2019)2024/06/20 21:41:29
(4, '歌登', 5, 2019)2024/06/20 21:41:32
(5, '中頓別', 1051, 2019)2024/06/20 21:41:36
(6, '豊富', 1054, 2019)2024/06/20 21:41:39
(7, '沼川', 1203, 2019)2024/06/20 21:41:42
(8, '宗谷岬', 1284, 2019)2024/06/20 21:41:46
(9, '浜鬼志別', 1285, 2019)2024/06/20 21:41:49
(10, '本泊', 1512, 2019)2024/06/20 21:41:53
(11, '声問', 1528, 2019)2024/06/20 21:41:56
(12, '礼文', 1546, 2019)2024/06/20 21:41:59
(13, '美深', 6, 2019)2024/06/20 21:42:03
(14, '名寄', 8, 2019)2024/06/20 21:42:06
(15, '下川', 10, 2019)2024/06/20 21:42:09
(16, '士別', 11, 2019)2024/06/20 21:42:13
(17, '上川', 15, 2019)2024/06/20 21:42:16
(18, '旭川', 47407, 2019)2024/06/20 21:42:19
(19, '富良野', 21, 2019)2024/06/20 21:42:23
(20, '朝日', 1047, 2019)2024/06/20 21:42:26
(21, '比布', 1048, 2019)2024/06/20 21:42:29
(22, '美瑛', 1052, 2019)2024/06/20 21:42:32
(23, '和寒', 1055, 2019)2024/06/20 21:

(194, '脇野沢', 1124, 2019)2024/06/20 21:52:03
(195, '青森大谷', 1495, 2019)2024/06/20 21:52:06
(196, '能代', 183, 2019)2024/06/20 21:52:10
(197, '鷹巣', 184, 2019)2024/06/20 21:52:13
(198, '鹿角', 185, 2019)2024/06/20 21:52:17
(199, '湯瀬', 186, 2019)2024/06/20 21:52:20
(200, '五城目', 188, 2019)2024/06/20 21:52:23
(201, '秋田', 47582, 2019)2024/06/20 21:52:26
(202, '岩見三内', 192, 2019)2024/06/20 21:52:30
(203, '角館', 193, 2019)2024/06/20 21:52:33
(204, '田沢湖', 194, 2019)2024/06/20 21:52:36
(205, '大曲', 195, 2019)2024/06/20 21:52:40
(206, '本荘', 196, 2019)2024/06/20 21:52:43
(207, '横手', 198, 2019)2024/06/20 21:52:46
(208, '象潟', 199, 2019)2024/06/20 21:52:50
(209, '矢島', 200, 2019)2024/06/20 21:52:53
(210, '湯沢', 202, 2019)2024/06/20 21:52:56
(211, '湯の岱', 204, 2019)2024/06/20 21:53:00
(212, '大館', 912, 2019)2024/06/20 21:53:03
(213, '大潟', 1035, 2019)2024/06/20 21:53:06
(214, '男鹿', 1036, 2019)2024/06/20 21:53:10
(215, '八森', 1043, 2019)2024/06/20 21:53:13
(216, '大正寺', 1050, 2019)2024/06/20 21:53:16
(217, '東由利', 1117

(385, '八王子', 366, 2019)2024/06/20 22:02:39
(386, '東京', 47662, 2019)2024/06/20 22:02:42
(387, '江戸川臨海', 370, 2019)2024/06/20 22:02:45
(388, '羽田', 371, 2019)2024/06/20 22:02:48
(389, '大島', 47675, 2019)2024/06/20 22:02:51
(390, '三宅島', 47677, 2019)2024/06/20 22:02:55
(391, '八丈島', 47678, 2019)2024/06/20 22:02:58
(392, '青梅', 1001, 2019)2024/06/20 22:03:01
(393, '練馬', 1002, 2019)2024/06/20 22:03:04
(394, '府中', 1133, 2019)2024/06/20 22:03:08
(395, '父島', 47971, 2019)2024/06/20 22:03:11
(396, '大島北ノ山', 1467, 2019)2024/06/20 22:03:14
(397, '八重見ヶ原', 1468, 2019)2024/06/20 22:03:17
(398, '三宅坪田', 1506, 2019)2024/06/20 22:03:20
(399, '新島', 1522, 2019)2024/06/20 22:03:24
(400, '神津島', 1523, 2019)2024/06/20 22:03:27
(401, '南鳥島', 47991, 2019)2024/06/20 22:03:30
(402, '香取', 375, 2019)2024/06/20 22:03:33
(403, '我孫子', 376, 2019)2024/06/20 22:03:37
(404, '成田', 378, 2019)2024/06/20 22:03:40
(405, '銚子', 47648, 2019)2024/06/20 22:03:43
(406, '千葉', 47682, 2019)2024/06/20 22:03:46
(407, '茂原', 381, 2019)2024/06/20 22

(575, '栢野', 571, 2019)2024/06/20 22:13:03
(576, '白山吉野', 973, 2019)2024/06/20 22:13:06
(577, '小松', 1324, 2019)2024/06/20 22:13:10
(578, '三井', 1544, 2019)2024/06/20 22:13:13
(579, '福井', 47616, 2019)2024/06/20 22:13:16
(580, '大野', 573, 2019)2024/06/20 22:13:19
(581, '今庄', 577, 2019)2024/06/20 22:13:23
(582, '敦賀', 47631, 2019)2024/06/20 22:13:26
(583, '小浜', 579, 2019)2024/06/20 22:13:29
(584, '美浜', 1010, 2019)2024/06/20 22:13:32
(585, '三国', 1071, 2019)2024/06/20 22:13:36
(586, '勝山', 1226, 2019)2024/06/20 22:13:39
(587, '越廼', 1316, 2019)2024/06/20 22:13:42
(588, '春江', 1537, 2019)2024/06/20 22:13:46
(589, '今津', 580, 2019)2024/06/20 22:13:49
(590, '彦根', 47761, 2019)2024/06/20 22:13:52
(591, '大津', 586, 2019)2024/06/20 22:13:55
(592, '土山', 587, 2019)2024/06/20 22:13:59
(593, '東近江', 963, 2019)2024/06/20 22:14:02
(594, '信楽', 964, 2019)2024/06/20 22:14:05
(595, '長浜', 976, 2019)2024/06/20 22:14:08
(596, '南小松', 1083, 2019)2024/06/20 22:14:12
(597, '米原', 1524, 2019)2024/06/20 22:14:15
(598, '宮津', 589

(766, '秋吉台', 769, 2019)2024/06/20 22:23:38
(767, '広瀬', 771, 2019)2024/06/20 22:23:41
(768, '豊田', 772, 2019)2024/06/20 22:23:44
(769, '山口', 47784, 2019)2024/06/20 22:23:48
(770, '防府', 775, 2019)2024/06/20 22:23:51
(771, '下松', 776, 2019)2024/06/20 22:23:54
(772, '下関', 47762, 2019)2024/06/20 22:23:58
(773, '宇部', 778, 2019)2024/06/20 22:24:01
(774, '安下庄', 779, 2019)2024/06/20 22:24:04
(775, '玖珂', 913, 2019)2024/06/20 22:24:08
(776, '油谷', 939, 2019)2024/06/20 22:24:11
(777, '岩国', 940, 2019)2024/06/20 22:24:14
(778, '柳井', 942, 2019)2024/06/20 22:24:18
(779, '八幡', 780, 2019)2024/06/20 22:24:21
(780, '行橋', 782, 2019)2024/06/20 22:24:24
(781, '飯塚', 47809, 2019)2024/06/20 22:24:28
(782, '前原', 785, 2019)2024/06/20 22:24:31
(783, '福岡', 47807, 2019)2024/06/20 22:24:34
(784, '朝倉', 788, 2019)2024/06/20 22:24:38
(785, '久留米', 790, 2019)2024/06/20 22:24:41
(786, '黒木', 792, 2019)2024/06/20 22:24:44
(787, '大牟田', 793, 2019)2024/06/20 22:24:47
(788, '宗像', 943, 2019)2024/06/20 22:24:51
(789, '添田', 1046, 2019

In [122]:
#2021年用コード
start_time_2021 = datetime.date(2021, 1, 1)
end_time_2021   = datetime.date(2021, 12, 31)

#現在の時刻を表示
now=time.strftime('%Y/%m/%d %H:%M:%S')
print(now)

Temp_df = pd.DataFrame(index=[], columns=['地点コード','地点','地点カナ','都道府県番号','区分','月','緯度','経度','降水量合計','平均気温','平均湿度','平均風速','日照時間合計','降雪日数'])
Temp_df_2021 = Temp_df.copy()
Temp_df_copy1 = Temp_df.copy()
Temp_df_copy2 = Temp_df.copy()

#2021_1から2021_2までのデータフレーム
for i in range(len(obs_stations)):
    Temp_df_copy2 = Temp_df_copy1.copy()
    month = start_time_2021
    while month < end_time_2021:
        Temp_df_copy2.loc[month,'地点コード']=obs_stations['地点コード'][i]
        Temp_df_copy2.loc[month,'地点']=obs_stations['地点'][i]
        Temp_df_copy2.loc[month,'地点カナ']=obs_stations['地点カナ'][i]
        Temp_df_copy2.loc[month,'都道府県番号']=obs_stations['府県番号'][i]
        Temp_df_copy2.loc[month,'区分']=obs_stations['区分'][i]
        Temp_df_copy2.loc[month,'月']=month.month
        Temp_df_copy2.loc[month,'緯度']=obs_stations['緯度'][i]
        Temp_df_copy2.loc[month,'経度']=obs_stations['経度'][i]
        month += relativedelta(months=1)
    Temp_df_2021 = pd.concat([Temp_df_2021,Temp_df_copy2], axis=0,ignore_index=True) 


def str2float(weather_data):
    try:
        return float(weather_data)
    except:
        return -100

def safe_split_string(element, default='NaN'):
    if element is not None and element.string is not None:
        return element.string.split(' ')[0]
    else:
        return default

# 各観測所のデータを取得
Temp_df_result_2021 = Temp_df_2021.copy()

num = 0
for index, value in obs_stations.iterrows():
    date_month = start_time_2021

    url = "https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_%s1.php?prec_no=%d&block_no=%04d&year=%d&month=&day=&view=" \
        % (str.lower(obs_stations['区分'].loc[index]), value['府県番号'], value['地点コード'], date_month.year)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find("table", {"class": "data2_s"})

    if trs is None:
        continue

    print(f"\r{index, obs_stations['地点'].loc[index], obs_stations['地点コード'].loc[index], date_month.year}", end="")
    if obs_stations['区分'].loc[index] == 'S':
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2021.loc[num, '降水量合計'] = str2float(safe_split_string(tds[3]))
            Temp_df_result_2021.loc[num, '平均気温'] = str2float(safe_split_string(tds[7]))
            Temp_df_result_2021.loc[num, '平均湿度'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2021.loc[num, '平均風速'] = str2float(safe_split_string(tds[14]))
            Temp_df_result_2021.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[19]))
            Temp_df_result_2021.loc[num, '降雪日数'] = str2float(safe_split_string(tds[25]))
            date_month += relativedelta(months=1)
            num += 1
    else:
        for tr in trs.findAll('tr')[3:]:
            tds = tr.findAll('td')
            Temp_df_result_2021.loc[num, '降水量合計'] = str2float(safe_split_string(tds[1]))
            Temp_df_result_2021.loc[num, '平均気温'] = str2float(safe_split_string(tds[5]))
            Temp_df_result_2021.loc[num, '平均湿度'] = str2float(safe_split_string(tds[10]))
            Temp_df_result_2021.loc[num, '平均風速'] = str2float(safe_split_string(tds[12]))
            Temp_df_result_2021.loc[num, '日照時間合計'] = str2float(safe_split_string(tds[17]))
            Temp_df_result_2021.loc[num, '降雪日数'] = None
            date_month += relativedelta(months=1)
            num += 1
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    time.sleep(3)

print(Temp_df_result_2021)

Temp_202101_202112_by_month=Temp_df_result_2021

Temp_202101_202112_by_month.to_csv("2021_by_month_data.csv", encoding="shift_jis",
                      index=False,header=True)

2024/06/20 23:08:33
(0, '稚内', 47401, 2021)2024/06/20 23:08:46
(1, '沓形', 2, 2021)2024/06/20 23:08:49
(2, '浜頓別', 3, 2021)2024/06/20 23:08:53
(3, '北見枝幸', 47402, 2021)2024/06/20 23:08:56
(4, '歌登', 5, 2021)2024/06/20 23:08:59
(5, '中頓別', 1051, 2021)2024/06/20 23:09:03
(6, '豊富', 1054, 2021)2024/06/20 23:09:07
(7, '沼川', 1203, 2021)2024/06/20 23:09:10
(8, '宗谷岬', 1284, 2021)2024/06/20 23:09:13
(9, '浜鬼志別', 1285, 2021)2024/06/20 23:09:17
(10, '本泊', 1512, 2021)2024/06/20 23:09:20
(11, '声問', 1528, 2021)2024/06/20 23:09:24
(12, '礼文', 1546, 2021)2024/06/20 23:09:27
(13, '美深', 6, 2021)2024/06/20 23:09:31
(14, '名寄', 8, 2021)2024/06/20 23:09:34
(15, '下川', 10, 2021)2024/06/20 23:09:38
(16, '士別', 11, 2021)2024/06/20 23:09:41
(17, '上川', 15, 2021)2024/06/20 23:09:44
(18, '旭川', 47407, 2021)2024/06/20 23:09:48
(19, '富良野', 21, 2021)2024/06/20 23:09:51
(20, '朝日', 1047, 2021)2024/06/20 23:09:54
(21, '比布', 1048, 2021)2024/06/20 23:09:58
(22, '美瑛', 1052, 2021)2024/06/20 23:10:01
(23, '和寒', 1055, 2021)2024/06/20 23:

(194, '脇野沢', 1124, 2021)2024/06/20 23:19:25
(195, '青森大谷', 1495, 2021)2024/06/20 23:19:28
(196, '能代', 183, 2021)2024/06/20 23:19:31
(197, '鷹巣', 184, 2021)2024/06/20 23:19:35
(198, '鹿角', 185, 2021)2024/06/20 23:19:38
(199, '湯瀬', 186, 2021)2024/06/20 23:19:41
(200, '五城目', 188, 2021)2024/06/20 23:19:45
(201, '秋田', 47582, 2021)2024/06/20 23:19:48
(202, '岩見三内', 192, 2021)2024/06/20 23:19:51
(203, '角館', 193, 2021)2024/06/20 23:19:54
(204, '田沢湖', 194, 2021)2024/06/20 23:19:57
(205, '大曲', 195, 2021)2024/06/20 23:20:01
(206, '本荘', 196, 2021)2024/06/20 23:20:04
(207, '横手', 198, 2021)2024/06/20 23:20:07
(208, '象潟', 199, 2021)2024/06/20 23:20:11
(209, '矢島', 200, 2021)2024/06/20 23:20:14
(210, '湯沢', 202, 2021)2024/06/20 23:20:17
(211, '湯の岱', 204, 2021)2024/06/20 23:20:21
(212, '大館', 912, 2021)2024/06/20 23:20:24
(213, '大潟', 1035, 2021)2024/06/20 23:20:27
(214, '男鹿', 1036, 2021)2024/06/20 23:20:31
(215, '八森', 1043, 2021)2024/06/20 23:20:34
(216, '大正寺', 1050, 2021)2024/06/20 23:20:37
(217, '東由利', 1117

(385, '八王子', 366, 2021)2024/06/20 23:29:55
(386, '東京', 47662, 2021)2024/06/20 23:29:58
(387, '江戸川臨海', 370, 2021)2024/06/20 23:30:02
(388, '羽田', 371, 2021)2024/06/20 23:30:05
(389, '大島', 47675, 2021)2024/06/20 23:30:08
(390, '三宅島', 47677, 2021)2024/06/20 23:30:11
(391, '八丈島', 47678, 2021)2024/06/20 23:30:15
(392, '青梅', 1001, 2021)2024/06/20 23:30:18
(393, '練馬', 1002, 2021)2024/06/20 23:30:21
(394, '府中', 1133, 2021)2024/06/20 23:30:24
(395, '父島', 47971, 2021)2024/06/20 23:30:28
(396, '大島北ノ山', 1467, 2021)2024/06/20 23:30:31
(397, '八重見ヶ原', 1468, 2021)2024/06/20 23:30:34
(398, '三宅坪田', 1506, 2021)2024/06/20 23:30:38
(399, '新島', 1522, 2021)2024/06/20 23:30:41
(400, '神津島', 1523, 2021)2024/06/20 23:30:44
(401, '南鳥島', 47991, 2021)2024/06/20 23:30:47
(402, '香取', 375, 2021)2024/06/20 23:30:50
(403, '我孫子', 376, 2021)2024/06/20 23:30:54
(404, '成田', 378, 2021)2024/06/20 23:30:57
(405, '銚子', 47648, 2021)2024/06/20 23:31:00
(406, '千葉', 47682, 2021)2024/06/20 23:31:04
(407, '茂原', 381, 2021)2024/06/20 23

(575, '栢野', 571, 2021)2024/06/20 23:40:18
(576, '白山吉野', 973, 2021)2024/06/20 23:40:22
(577, '小松', 1324, 2021)2024/06/20 23:40:25
(578, '三井', 1544, 2021)2024/06/20 23:40:28
(579, '福井', 47616, 2021)2024/06/20 23:40:32
(580, '大野', 573, 2021)2024/06/20 23:40:35
(581, '今庄', 577, 2021)2024/06/20 23:40:38
(582, '敦賀', 47631, 2021)2024/06/20 23:40:41
(583, '小浜', 579, 2021)2024/06/20 23:40:45
(584, '美浜', 1010, 2021)2024/06/20 23:40:48
(585, '三国', 1071, 2021)2024/06/20 23:40:52
(586, '勝山', 1226, 2021)2024/06/20 23:40:55
(587, '越廼', 1316, 2021)2024/06/20 23:40:58
(588, '春江', 1537, 2021)2024/06/20 23:41:01
(589, '今津', 580, 2021)2024/06/20 23:41:04
(590, '彦根', 47761, 2021)2024/06/20 23:41:08
(591, '大津', 586, 2021)2024/06/20 23:41:11
(592, '土山', 587, 2021)2024/06/20 23:41:14
(593, '東近江', 963, 2021)2024/06/20 23:41:17
(594, '信楽', 964, 2021)2024/06/20 23:41:21
(595, '長浜', 976, 2021)2024/06/20 23:41:24
(596, '南小松', 1083, 2021)2024/06/20 23:41:27
(597, '米原', 1524, 2021)2024/06/20 23:41:30
(598, '宮津', 589

(766, '秋吉台', 769, 2021)2024/06/20 23:50:46
(767, '広瀬', 771, 2021)2024/06/20 23:50:49
(768, '豊田', 772, 2021)2024/06/20 23:50:52
(769, '山口', 47784, 2021)2024/06/20 23:50:55
(770, '防府', 775, 2021)2024/06/20 23:50:59
(771, '下松', 776, 2021)2024/06/20 23:51:02
(772, '下関', 47762, 2021)2024/06/20 23:51:05
(773, '宇部', 778, 2021)2024/06/20 23:51:08
(774, '安下庄', 779, 2021)2024/06/20 23:51:12
(775, '玖珂', 913, 2021)2024/06/20 23:51:15
(776, '油谷', 939, 2021)2024/06/20 23:51:18
(777, '岩国', 940, 2021)2024/06/20 23:51:21
(778, '柳井', 942, 2021)2024/06/20 23:51:25
(779, '八幡', 780, 2021)2024/06/20 23:51:28
(780, '行橋', 782, 2021)2024/06/20 23:51:31
(781, '飯塚', 47809, 2021)2024/06/20 23:51:34
(782, '前原', 785, 2021)2024/06/20 23:51:38
(783, '福岡', 47807, 2021)2024/06/20 23:51:41
(784, '朝倉', 788, 2021)2024/06/20 23:51:44
(785, '久留米', 790, 2021)2024/06/20 23:51:48
(786, '黒木', 792, 2021)2024/06/20 23:51:51
(787, '大牟田', 793, 2021)2024/06/20 23:51:54
(788, '宗像', 943, 2021)2024/06/20 23:51:58
(789, '添田', 1046, 2021

In [224]:
data2020 = pd.read_csv("2020_by_month_data.csv",encoding='shift-jis')
data2019 = pd.read_csv("2019_by_month_data.csv",encoding='shift-jis')
data2021 = pd.read_csv("2021_by_month_data.csv",encoding='shift-jis')

#同じ地点コードの有無の確認
count = data2020.groupby('地点コード').count()
#count = data2021.groupby('地点コード').count()
#count = data2019.groupby('地点コード').count()

# 'counts' データフレームからデータ数が12よりも多い地域を抽出
more_than_12 = count[count['月'] > 12]
more_than_12.index[0]
#print(more_than_12['地点コード'])
#print(data2020[data2020['地点コード']==more_than_12.index[0]]) #都道府県コード49:山梨　50:静岡

#静岡の富士山データを落とす
data2020.drop(data2020[(data2020['地点コード'] == more_than_12.index[0]) & (data2020['都道府県番号'] == 50)].index, inplace=True)
data2021.drop(data2021[(data2021['地点コード'] == more_than_12.index[0]) & (data2021['都道府県番号'] == 50)].index, inplace=True)
data2019.drop(data2019[(data2019['地点コード'] == more_than_12.index[0]) & (data2019['都道府県番号'] == 50)].index, inplace=True)
#indexの振り直し
data2020.reset_index(drop=True, inplace=True)
data2021.reset_index(drop=True, inplace=True)
data2019.reset_index(drop=True, inplace=True)

In [225]:
# '2020-01' から '2020-12' までの年月を生成
times_2020 = pd.date_range(start='2020-01', end='2021-01', freq='M').strftime('%Y-%m')
times_list_2020 = []

times_2021 = pd.date_range(start='2021-01', end='2022-01', freq='M').strftime('%Y-%m')
times_list_2021 = []

times_2019 = pd.date_range(start='2019-01', end='2020-01', freq='M').strftime('%Y-%m')
times_list_2019 = []

for _ in range(len(data2020['地点コード'].unique())):
    times_list_2020.extend(times_2020)
    
for _ in range(len(data2021['地点コード'].unique())):
    times_list_2021.extend(times_2021)

for _ in range(len(data2019['地点コード'].unique())):
    times_list_2019.extend(times_2019)

data2020['年月'] = times_list_2020
data2021['年月'] = times_list_2021
data2019['年月'] = times_list_2019

In [226]:

# Filter the DataFrame for rows where '地点' is either '稚内' or '沓形'
df2 = data2020[data2020['地点'].isin(['稚内', '沓形'])]
df2

,地点,月,緯度,経度,降水量合計,平均気温,平均湿度,平均風速,日照時間合計,降雪日数,地点コード,地点カナ,都道府県番号,区分,年月
0,稚内,1,45.4150,141.6783,35.0,-3.7,71.0,4.8,62.2,26.0,47401.0,ワツカナイ,11.0,S,2020-01
1,稚内,2,45.4150,141.6783,39.5,-4.2,74.0,4.7,73.4,24.0,47401.0,ワツカナイ,11.0,S,2020-02
2,稚内,3,45.4150,141.6783,54.0,1.1,72.0,5.6,174.2,17.0,47401.0,ワツカナイ,11.0,S,2020-03
3,稚内,4,45.4150,141.6783,42.5,4.3,77.0,4.8,208.9,13.0,47401.0,ワツカナイ,11.0,S,2020-04
4,稚内,5,45.4150,141.6783,63.0,9.9,83.0,5.0,207.6,0.0,47401.0,ワツカナイ,11.0,S,2020-05
5,稚内,6,45.4150,141.6783,98.0,13.9,87.0,4.3,114.9,0.0,47401.0,ワツカナイ,11.0,S,2020-06
6,稚内,7,45.4150,141.6783,68.5,17.7,87.0,4.6,194.4,0.0,47401.0,ワツカナイ,11.0,S,2020-07
7,稚内,8,45.4150,141.6783,332.0,19.7,86.0,4.7,132.0,0.0,47401.0,ワツカナイ,11.0,S,2020-08
8,稚内,9,45.4150,141.6783,112.5,18.5,80.0,5.1,149.1,0.0,47401.0,ワツカナイ,11.0,S,2020-09
9,稚内,10,45.4150,141.6783,168.0,12.1,68.0,4.3,133.2,0.0,47401.0,ワツカナイ,11.0,S,2020-10


In [231]:
# 稚内のデータのみをフィルタリング
df_wakkanai = df2[df2['地点'] == '沓形']

# 3ヶ月移動平均を計算
df_wakkanai['平均気温_移動平均'] = df_wakkanai['降雪日数'].rolling(window=3).mean()
df_wakkanai['平均気温_移動平均']

# 対象とする数値列のリスト
numeric_columns = ['降水量合計', '平均気温',]

# 各地点ごとに数値列ごとに3ヶ月移動平均を計算
for column in numeric_columns:
    df2[f'{column}_移動平均'] = df2.groupby('地点')[column].rolling(window=3).mean().reset_index(level=0, drop=True)

df2

<ipython-input-231-e96518986e5d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wakkanai['平均気温_移動平均'] = df_wakkanai['降雪日数'].rolling(window=3).mean()
<ipython-input-231-e96518986e5d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[f'{column}_移動平均'] = df2.groupby('地点')[column].rolling(window=3).mean().reset_index(level=0, drop=True)
<ipython-input-231-e96518986e5d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,地点,月,緯度,経度,降水量合計,平均気温,平均湿度,平均風速,日照時間合計,降雪日数,地点コード,地点カナ,都道府県番号,区分,年月,降水量合計_移動平均,平均気温_移動平均
0,稚内,1,45.4150,141.6783,35.0,-3.7,71.0,4.8,62.2,26.0,47401.0,ワツカナイ,11.0,S,2020-01,NaN,NaN
1,稚内,2,45.4150,141.6783,39.5,-4.2,74.0,4.7,73.4,24.0,47401.0,ワツカナイ,11.0,S,2020-02,NaN,NaN
2,稚内,3,45.4150,141.6783,54.0,1.1,72.0,5.6,174.2,17.0,47401.0,ワツカナイ,11.0,S,2020-03,42.833333,-2.266667
3,稚内,4,45.4150,141.6783,42.5,4.3,77.0,4.8,208.9,13.0,47401.0,ワツカナイ,11.0,S,2020-04,45.333333,0.400000
4,稚内,5,45.4150,141.6783,63.0,9.9,83.0,5.0,207.6,0.0,47401.0,ワツカナイ,11.0,S,2020-05,53.166667,5.100000
5,稚内,6,45.4150,141.6783,98.0,13.9,87.0,4.3,114.9,0.0,47401.0,ワツカナイ,11.0,S,2020-06,67.833333,9.366667
6,稚内,7,45.4150,141.6783,68.5,17.7,87.0,4.6,194.4,0.0,47401.0,ワツカナイ,11.0,S,2020-07,76.500000,13.833333
7,稚内,8,45.4150,141.6783,332.0,19.7,86.0,4.7,132.0,0.0,47401.0,ワツカナイ,11.0,S,2020-08,166.166667,17.100000
8,稚内,9,45.4150,141.6783,112.5,18.5,80.0,5.1,149.1,0.0,47401.0,ワツカナイ,11.0,S,2020-09,171.000000,18.633333
9,稚内,10,45.4150,141.6783,168.0,12.1,68.0,4.3,133.2,0.0,47401.0,ワツカナイ,11.0,S,2020-10,204.166667,16.766667


In [282]:
#データの縦結合
df_con = pd.concat([data2019, data2020,data2021],ignore_index=True)
df_con

# 対象とする数値列のリスト
col = ['降水量合計', '平均気温', '平均湿度', '平均風速', '日照時間合計','降雪日数']

# 新しいデータフレームを作成し、元の識別情報をコピー
moving_ave_df = df_con[['地点', '月', '緯度', '経度', '地点コード', '地点カナ', '都道府県番号', '区分', '年月']].copy()

# 新しい変数「期間」を作成
moving_ave_df['期間'] = moving_ave_df['月'].apply(lambda x: f"{(x - 2) % 12 or 12}～{x}")

# 各数値列に対して移動平均を計算し、新しいデータフレームに追加

for column in col:
    moving_ave_df[f'{column}_移動平均'] = df_con.groupby('地点コード')[column].rolling(window=3).mean().reset_index(level=0, drop=True)

moving_ave_df

moving_ave_df.to_csv("2019-2021_by_3months.csv", encoding="UTF-8",
                      index=False,header=True)
moving_ave_df_2020 = moving_ave_df[moving_ave_df['年月'].str[:4]=='2020']

moving_ave_df_2020.to_csv("2020_by_3months.csv", encoding="UTF-8",
                      index=False,header=True)